<a href="https://colab.research.google.com/github/caugustocbastos/app-vai-passar-onde/blob/main/Projeto_Final_VaiPassarOnde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Instalação das Bibliotecas ---

%pip -q install google-genai
%pip install -q google-adk
%pip install -q requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [23]:
# --- Imports e Configuração --
import re
import json
import os
import warnings
import textwrap
from datetime import date, timedelta, datetime, timezone
import unicodedata # Importa para normalização de strings (acentos)
from IPython.display import display, Markdown

from google.adk.agents import Agent
from google import genai
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

warnings.filterwarnings("ignore")

# Configuração da API Key
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception:
    os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", "YOUR_API_KEY") # Substitua YOUR_API_KEY ou defina a variável de ambiente

# --- Funções Auxiliares ---

def normalize_string(s):
    """Remove acentos e converte para minúsculas."""
    if not s:
        return ""
    s = str(s)
    nfkd_form = unicodedata.normalize('NFKD', s)
    only_ascii = nfkd_form.encode('ascii', 'ignore').decode('utf-8')
    return only_ascii.lower()

def call_agent(agent: Agent, message_text: str) -> str:
    """
    Chama um agente com uma mensagem e retorna sua resposta final em texto.
    Trata erros básicos durante a execução do agente.
    """
    session_service = InMemorySessionService()
    # Usar um session_id único por chamada para evitar cache indesejado entre chamadas distintas
    session_id = f"{agent.name}_{datetime.now().timestamp()}"
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id=session_id)
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id=session_id, new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
    except Exception as e:
        print(f"Erro durante a execução do agente {agent.name}: {e}")
        final_response = ""

    return final_response.strip()

def to_markdown(text):
    """Formata texto com Markdown para exibição no Notebook."""
    # Remove espaços duplos antes de bullet points
    text = re.sub(r'\s+[*•]\s*', '• ', text)
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def parse_agent_json_response(raw_response_str: str, agent_name: str = "Agente"):
    """
    Tenta extrair JSON de uma string de resposta de agente,
    tratando blocos markdown ```json ``` e erros de parsing.
    """
    json_string_to_parse = raw_response_str.strip()
    parsed_data = []

    # Tenta encontrar e extrair o JSON dentro de um bloco markdown ```json ```
    start_index = json_string_to_parse.find("```json")
    end_index = json_string_to_parse.find("```", start_index + 7)

    if start_index != -1 and end_index != -1:
        json_string_to_parse = json_string_to_parse[start_index + 7 : end_index].strip()

    try:
        parsed_data = json.loads(json_string_to_parse)

    except json.JSONDecodeError as e:
        print(f"--- ❌ Erro ao parsear JSON do {agent_name}: {e}.")
        # print(f"String sendo parseada (parcial):\\n{json_string_to_parse[:500]}...\n") # Debug
        parsed_data = []
    except Exception as e:
        print(f"--- ❌ Erro inesperado no parse do {agent_name}: {e}")
        parsed_data = []

    # Garantir que o retorno é uma lista, mesmo que o agente retorne um único objeto JSON
    if isinstance(parsed_data, dict):
        parsed_data = [parsed_data]

    return parsed_data

def is_unwanted_league(competition_name):
    """Verifica se a competição está na lista de ligas a serem removidas."""
    if not competition_name:
        return False
    normalized_comp = normalize_string(competition_name)
    unwanted_keywords = [
        "serie b", "serie c", "serie d", # Brazilian lower divisions
        "liga 2", "segunda liga", "bundesliga 2", "serie b tim", # European lower divisions
        "feminina", "women's", "femenino", # Women's leagues
        "sub-15", "sub-17", "sub-20", "sub-23" # Youth leagues
        # Adicione outras keywords conforme necessário
    ]
    for keyword in unwanted_keywords:
        if keyword in normalized_comp:
            return True
    return False

def format_favorite_game_line(game_data):
    """Formata uma única linha para a próxima partida do favorito com emojis."""
    if not game_data:
        return "Informação da próxima partida indisponível."

    comp = game_data.get('competition', 'Competição Indefinida')
    date_str = game_data.get('date', 'Data Indefinida')
    time_str = game_data.get('time', 'Horário Indefinido')
    home = game_data.get('home_team', 'Time Casa Indefinido')
    away = game_data.get('away_team', 'Time Fora Indefinido')
    channels = ", ".join(game_data.get('channels', ['Sem transmissão']))

    # Usa o formato pedido no GeradorTexto e adiciona emojis antes
    formatted_line = f"⭐⚽ {comp} - Dia {date_str} - {time_str} - {home} X {away} - Canais: {channels}"

    return formatted_line


# --- Definição dos Agentes ---
# Mantidos como estavam no código original

class AgenteColetorJogos(Agent):
    def __init__(self):
         super().__init__(
             name="coletor_jogos",
             model="gemini-2.0-flash",
             instruction="""
Você é especialista em buscar dados basicos de jogos de futebol e suas transmissões no Brasil.
Priorize as ligas masculinas de futebol.
Priorize os times brasileiros.
Reduza informações dos times que não estão no Brasil.
Dos times europeus liste apenas jogos dos principais 20 principais times segundo o ranking da FIFA.
Use Obrigatoriamente ferramentas (Google Search) em sites de esportes e guias de programação confiáveis no Brasil.
Para cada jogo encontrado, colete OBRIGATORIAMENTE:
Data (DD/MM/AAAA), Horário de Brasilia (HH:MM ou HHhMM), Nome Time Casa (ou País), Nome Time Fora (ou País),
Nome COMPLETO Competição/Liga/Copa, Info país/região/confederação (Ex: "Brasil", "Inglaterra", "CONMEBOL", "FIFA Global"),
Tipo: "Clube" ou "Seleção".
Lista de canais/plataformas que transmitirão o jogo NO BRASIL.

Retorne obrigatoriamente a lista de jogos como string JSON. Cada item é objeto JSON com chaves: date, time, home_team, away_team, competition, country_region, type, e channels (lista de strings).
Se não encontrar canais para um jogo, use `channels: ["Sem transmissão"]` ou liste os canais encontrados.
Se não encontrar jogos para o período e competições solicitados, retorne lista JSON vazia `[]`.
Remova qualquer outro texto, retorne estritamente a string JSON, idealmente formatada dentro de um bloco markdown ```json ```.
""",
             description="Busca e coleta dados brutos de jogos de futebol e suas transmissões no Brasil.",
             tools=[google_search]
            )

class AgenteProcessador(Agent):
     def __init__(self):
         super().__init__(
            name="processador_dados",
            model="gemini-2.0-flash",
            instruction="""
Você estrutura dados de jogos e marca o jogo do time do coração.
Receba string JSON: lista de jogos (com date, time, home_team, away_team, competition, country_region, type, channels).
Receba o nome do time do coração (clube, string, pode ser vazio).
Para cada jogo na lista, verifique se home_team ou away_team corresponde AO NOME DO TIME DO CORAÇÃO (se não vazio), ignorando MAIÚSCULAS/MINÚSCULAS e ACENTOS.
Adicione um campo booleano 'is_favorite_game': true se o jogo for do time do coração, false caso contrário.
Estruture CADA item no formato: {date, time, competition, home_team, away_team, channels, is_favorite_game}. Mantenha todos os outros campos originais se presentes.
Retorne a lista ESTRUTURADA com a flag 'is_favorite_game' adicionada como string JSON, idealmente formatada dentro de um bloco markdown ```json ```. Não altere a ordem da lista que recebeu. Não filtre por data.
""",
            description="Estrutura dados de jogos e marca o jogo do time do coração.",
            tools=[]
           )


class AgenteGeradorTexto(Agent):
     def __init__(self):
         super().__init__(
            name="gerador_texto",
            model="gemini-2.0-flash",
            instruction="""
Você formata texto final de jogos estilo WhatsApp.
Receba string JSON: lista de jogos ESTRUTURADA (cada item contém 'is_favorite_game' flag, date, time, competition, home_team, away_team, channels).
Converta lista em UMA string de texto puro para WhatsApp.
Para CADA ITEM, gere UMA linha no formato: [competition] - Dia DD/MM - HH:MM - [Time Casa/País] X [Time Fora/País] - Canais: [Lista de Canais ou "Sem transmissão"]. Pule uma linha após os Canais.
NÃO adicione cabeçalho (ex: "📺 Jogos de Hoje:"). O cabeçalho será adicionado pelo script principal.
Se a lista recebida for vazia, retorne uma string vazia.
""",
            description="Formata dados estruturados de jogos em texto para WhatsApp.",
            tools=[]
           )


# --- Lógica Principal de Orquestração (Python) ---

if __name__ == "__main__":
    # Calcular data e hora de hoje em Horário de Brasília (-03:00)
    now_utc = datetime.now(timezone.utc)
    brasilia_offset = timedelta(hours=-3)
    now_brasilia = now_utc.astimezone(timezone(brasilia_offset))

    data_de_hoje_str = now_brasilia.strftime("%d/%m/%Y")

    print("⚽ Iniciando o Sistema de Jogos de Futebol ⚽")

    # --- 1. Obter o Time do Coração ---
    time_coracao = input("❓ Digite o nome do seu time do coração (clube, ou deixe em branco): ").strip()
    normalized_favorite_team = normalize_string(time_coracao) # Normaliza o time informado

    if time_coracao:
        print(f"Ok, buscando informações do {time_coracao} e jogos de hoje.")
    else:
        print("Ok, buscando jogos para as competições principais.")

    # --- Instanciar os Agentes ---
    # Mantida a lista de competições para refinar buscas futuras, se necessário,
    # mas a busca inicial do favorito será mais ampla se o agente permitir.
    lista_de_competicoes_para_busca = """
    Premier League(somente os 6 principais times), La Liga(somente os 3 principais times), Serie A(somente os 4 principais times),, Borussia Dortmund, Bayern de Munique, Ligue 1(somente os 3 principais times),, Primeira Liga Portugal(somente os 3 principais times), Copa do Rei, FA Cup,
    Brasileirão Série A, Copa do Brasil, Copa Libertadores, Copa Sulamericana, Campeonato Brasileio Serie A
    Campeonato Saudita (somente os 3 principais times), Mundial de Clubes FIFA, Eliminatórias Copa do Mundo, Eurocopa, Copa América, Liga das Nações.
    """
    agente_coletor_jogos = AgenteColetorJogos()
    agente_processador = AgenteProcessador()
    agente_gerador_texto = AgenteGeradorTexto()

    # --- 2. Se o time do coração foi informado, buscar a PRÓXIMA partida dele primeiro ---
    next_favorite_game_info = None
    if normalized_favorite_team:
        print(f"\n--- 🔍 Buscando a próxima partida agendada de {time_coracao}... ---")
        # Prompt focado na próxima partida, independente da competição inicial listada
        coletor_prompt_next_fav = f"""
Busque a proxima partida agendada do time "{time_coracao}".
Inclua data (DD/MM/AAAA), horário de Brasilia (HH:MM ou HHhMM), competição e transmissão no Brasil.
Retorne APENAS a informação da PRIMEIRA partida encontrada como JSON.
"""
        next_fav_match_json_str = call_agent(agente_coletor_jogos, coletor_prompt_next_fav.strip())
        next_fav_match_raw = parse_agent_json_response(next_fav_match_json_str, f"Agente Coletor Jogos (Próx {time_coracao})")

        if next_fav_match_raw:
            # Processar apenas este jogo para marcar a flag is_favorite_game (será True)
            processed_next_fav_game_json_str = call_agent(agente_processador, f"Marque o time '{time_coracao}' no seguinte jogo (JSON): {json.dumps(next_fav_match_raw, ensure_ascii=False)}")
            processed_next_fav_game_list = parse_agent_json_response(processed_next_fav_game_json_str, "Agente Processador (Próx Favorito)")
            if processed_next_fav_game_list:
                 next_favorite_game_info = processed_next_fav_game_list[0] # Pega o único item processado
                 #print(f"--- ✅ Próxima partida de {time_coracao} encontrada. ---")

                 # --- Exibir a próxima partida do time do coração IMEDIATAMENTE ---
                 print("\n" + "="*40)
                 print(f"   ⭐ Próxima Partida do {time_coracao} ⭐") # Título com emoji
                 print("="*40)
                 # Usa a função auxiliar para formatar com emojis
                 formatted_fav_line = format_favorite_game_line(next_favorite_game_info)
                 display(to_markdown(formatted_fav_line))
                 print("-" * 40) # Separador visual após o jogo favorito

            else:
                print(f"--- ❌ Não foi possível processar a próxima partida de {time_coracao}. ---")
        else:
             print(f"--- ❌ Não foi possível encontrar a próxima partida para o {time_coracao}. ---")


    # --- 3. Chamar Agente Coletor para buscar jogos de HOJE ---
    # Pedir mais do que 15 jogos para garantir que teremos 15 relevantes após filtros
    print("\n--- 🔍 Buscando jogos e transmissões agendadas para HOJE... ---")
    # Mantido o foco nas competições listadas para a busca geral do dia
    coletor_prompt_hoje = f"""
Busque jogos agendados para HOJE ({data_de_hoje_str}) exclusivamente dos seguintes times e competições, respeitando o limite de times entre parenteses, INCLUINDO as transmissões no Brasil: {lista_de_competicoes_para_busca}.
Tente encontrar no minimo 40 jogos para hoje focando nos mais relevantes.
"""
    jogos_hoje_json_str = call_agent(agente_coletor_jogos, coletor_prompt_hoje.strip())

    # --- Parsing, Limpeza e Filtragem dos Jogos de HOJE ---
    cleaned_hoje_response = re.sub(r'\]\s*\[[\d,\s]*\]', r']', jogos_hoje_json_str) # Limpeza básica se JSONs vierem concatenados
    jogos_hoje_brutos = parse_agent_json_response(cleaned_hoje_response, "Agente Coletor Jogos (Hoje)")

    # Filtrar jogos de hoje pela data e ligas indesejadas
    jogos_hoje_filtrados = [
        jogo for jogo in jogos_hoje_brutos
        if jogo.get('date') == data_de_hoje_str and not is_unwanted_league(jogo.get('competition'))
    ]

    # Limitar aos 15 primeiros jogos encontrados
    jogos_hoje_top15 = jogos_hoje_filtrados[:20]

    #print(f"--- ✅ Encontrados {len(jogos_hoje_top15)} jogos para hoje (Top 15 após filtros). ---")

    # --- 4. Remover a próxima partida do favorito da lista de hoje, SE ela for hoje e já foi encontrada ---
    # Isso evita duplicidade se o jogo favorito for hoje E estiver no top 15
    jogos_hoje_para_processar = jogos_hoje_top15 # Começa com a lista completa do top 15 de hoje

    if next_favorite_game_info:
        next_fav_date = next_favorite_game_info.get('date')
        next_fav_time = next_favorite_game_info.get('time')
        next_fav_home = normalize_string(next_favorite_game_info.get('home_team', ''))
        next_fav_away = normalize_string(next_favorite_game_info.get('away_team', ''))

        # Verificar se a próxima partida encontrada é um jogo de hoje
        is_next_fav_game_today = (next_fav_date == data_de_hoje_str)

        if is_next_fav_game_today:
             print("--- 🗑️ O jogo do seu time é hoje!!")
             # Cria uma nova lista excluindo o jogo favorito
             jogos_hoje_sem_fav = []
             for jogo in jogos_hoje_top15:
                  jogo_date = jogo.get('date')
                  jogo_time = jogo.get('time')
                  jogo_home = normalize_string(jogo.get('home_team', ''))
                  jogo_away = normalize_string(jogo.get('away_team', ''))

                  # Compara data, hora e times (considera inversão home/away)
                  is_same_game = (
                      jogo_date == next_fav_date and
                      jogo_time == next_fav_time and
                      (
                         (jogo_home == next_fav_home and jogo_away == next_fav_away) or
                         (jogo_home == next_fav_away and jogo_away == next_fav_home)
                      )
                  )
                  if not is_same_game:
                      jogos_hoje_sem_fav.append(jogo)

             jogos_hoje_para_processar = jogos_hoje_sem_fav # Usa a lista dedupicada para o processamento e exibição


    # --- 5. Processar os Jogos de HOJE (lista sem o favorito se foi hoje) para marcar outros jogos do favorito se houver ---
    # Passamos o nome do time para o Processador para que ele marque outros jogos do mesmo time HOJE, caso existam.
    #print("\n--- ✨ Processando jogos de hoje... ---")
    # Note: O Processador ainda pode marcar outros jogos se o time do coração jogar mais de uma vez hoje
    # e esses outros jogos estiverem na lista jogos_hoje_para_processar.
    jogos_hoje_processed_json_str = call_agent(agente_processador, f"Marque o time '{time_coracao}' nos seguintes jogos (JSON): {json.dumps(jogos_hoje_para_processar, ensure_ascii=False)}")

    # --- Parsing da Resposta do Agente Processador (Hoje) ---
    jogos_hoje_processed = parse_agent_json_response(jogos_hoje_processed_json_str, "Agente Processador (Hoje)")

    if not jogos_hoje_processed and jogos_hoje_para_processar:
        #print("--- ⚠️ Aviso: O Agente Processador não retornou dados para os jogos de hoje. Exibindo lista bruta. ---")
        jogos_hoje_processed = jogos_hoje_para_processar # Fallback


    # --- 6. Exibir os Resultados ---

    #print("\n" + "="*40)
    #print("         ⚽ Agenda de Jogos") # Cabeçalho principal unificado
    #print("="*40)

    # A próxima partida do favorito já foi exibida logo após ser encontrada (passo 2/3)
    # Aqui, exibimos apenas a lista dos outros jogos de hoje.

    # Ordenar os jogos de hoje por horário antes de gerar o texto
    try:
        def parse_time_flexible(time_str):
            if not time_str: # Handle empty time string
                 return datetime.strptime('00:00', '%H:%M').time()
            if 'h' in time_str:
                time_str = time_str.replace('h', ':')
            try:
                return datetime.strptime(time_str, '%H:%M:%S').time()
            except ValueError:
                 try:
                    return datetime.strptime(time_str, '%H:%M').time()
                 except ValueError:
                     # Fallback mais robusto
                     #print(f"Warning: Não foi possível parsear o horário '{time_str}'. Usando 00:00.")
                     return datetime.strptime('00:00', '%H:%M').time()


        # Tenta ordenar mesmo se alguns não tiverem, tratando no parse_time_flexible
        jogos_hoje_processed.sort(key=lambda x: parse_time_flexible(x.get('time', '')))

    except Exception as e:
        #print(f"Warning: Erro inesperado ao ordenar jogos de hoje: {e}")
        pass # Continua sem ordenar se houver erro


    # Chamar Agente Gerador de Texto para formatar a lista de jogos de hoje
    if jogos_hoje_processed:
        # Certifica que a lista passada para o gerador é a lista de jogos de hoje processada
        # Passa time_coracao para o Gerador se ele tiver alguma lógica extra, ou ajuste o Gerador
        # para não depender dele se a marcação já foi feita pelo Processador.
        # Como a marcação já foi feita, o Gerador só precisa formatar a lista.
        texto_hoje_str = call_agent(agente_gerador_texto, f"Formate a seguinte lista ESTRUTURADA em texto estilo WhatsApp SEM CABEÇALHO:\\n{json.dumps(jogos_hoje_processed, ensure_ascii=False)}\n")
    else:
        texto_hoje_str = "" # String vazia se não houver jogos processados


    print("\n📺 Jogos de Hoje:") # Cabeçalho para os jogos de hoje

    if texto_hoje_str:
        display(to_markdown(texto_hoje_str.strip()))
    else:
         # Esta mensagem só aparece se não há jogos hoje na lista (após deduplicação e processamento)
         print("Nenhum jogo encontrado para hoje nas competições principais.")


    print("\n" + "="*40)
    print("Processo de busca de jogos encerrado.")
    print("="*40)

⚽ Iniciando o Sistema de Jogos de Futebol ⚽
❓ Digite o nome do seu time do coração (clube, ou deixe em branco): gremio
Ok, buscando informações do gremio e jogos de hoje.

--- 🔍 Buscando a próxima partida agendada de gremio... ---

   ⭐ Próxima Partida do gremio ⭐


> ⭐⚽ Campeonato Brasileiro - Dia 17/05/2025 - 21:00 - São Paulo X Grêmio - Canais: SporTV, Premiere

----------------------------------------

--- 🔍 Buscando jogos e transmissões agendadas para HOJE... ---
--- 🗑️ O jogo do seu time é hoje!!

📺 Jogos de Hoje:


> Bundesliga - Dia 17/05 - 10:30 - Hoffenheim X Bayern de Munique - Canais: SporTV, CazéTV, OneFootball, RedeTV!
> 
> Bundesliga - Dia 17/05 - 10:30 - Borussia Dortmund X Holstein Kiel - Canais: TV Cultura, Canal GOAT, OneFootball
> 
> Bundesliga - Dia 17/05 - 10:30 - Mainz X Bayer Leverkusen - Canais: Nosso Futebol, Amazon Prime Video, CazéTV, OneFootball
> 
> Bundesliga - Dia 17/05 - 10:30 - Borussia Mönchengladbach X Wolfsburg - Canais: SporTV 2, Canal GOAT, OneFootball
> 
> FA Cup - Dia 17/05 - 12:30 - Crystal Palace X Manchester City - Canais: ESPN, Disney+
> 
> Brasileirão Série A - Dia 17/05 - 16:00 - Ceará X Sport - Canais: Premiere
> 
> Campeonato Francês - Dia 17/05 - 16:00 - Lille X Reims - Canais: Amazon Prime Video, CazéTV
> 
> Campeonato Francês - Dia 17/05 - 16:00 - PSG X Auxerre - Canais: CazéTV
> 
> Campeonato Francês - Dia 17/05 - 16:00 - Saint-Étienne X Toulouse - Canais: CazéTV
> 
> Brasileirão Feminino - Dia 17/05 - 17:00 - Cruzeiro (F) X Ferroviária (F) - Canais: TV Brasil, SporTV
> 
> Brasileirão Série A - Dia 17/05 - 18:30 - Vasco X Fortaleza - Canais: Record, CazéTV, Premiere
> 
> Brasileirão Série A - Dia 17/05 - 21:00 - São Paulo X Grêmio - Canais: SporTV, Premiere, Amazon Prime Video
> 
> Brasileirão Feminino - Dia 17/05 - 21:00 - Corinthians (F) X Sport (F) - Canais: TV Brasil, SporTV


Processo de busca de jogos encerrado.
